<a href="https://colab.research.google.com/github/ye2x/WebAuto_Selenium/blob/main/scraping_review_on_goodread_with_google_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 23.6 MB/s eta 0:00:00


In [2]:
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
# Input Alamat Web yang menuju riview
link = "https://www.goodreads.com/book/show/325779/reviews"

# Fungsi untuk inisialisasi WebDriver
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def click_expand_buttons_until_gone(driver, expand_button_xpath):
    while True:
        try:
            # Tunggu tombol "Expand" tersedia (jika ada)
            expand_button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, expand_button_xpath))
            )

            # Scroll ke elemen untuk memastikan terlihat
            driver.execute_script("arguments[0].scrollIntoView(true);", expand_button)
            time.sleep(1)

            # Klik tombol "Expand" menggunakan ActionChains
            actions = ActionChains(driver)
            actions.move_to_element(expand_button).click().perform()
            print("Tombol Expand berhasil diklik.")

            # Tunggu sebentar untuk memastikan konten dimuat
            time.sleep(2)

        except Exception as e:
            # Jika tombol "Expand" tidak ditemukan lagi, keluar dari loop
            # print("Tidak ada tombol Expand yang tersisa atau terjadi kesalahan:", e)
            break

# Fungsi utama
def main():
    # Inisialisasi WebDriver dan buka URL
    driver = web_driver()
    url = link
    driver.get(url)
    time.sleep(5)

    # XPath tombol "Expand"
    expand_button_xpath = "/html/body/div[1]/div[2]/main/div[1]/div[2]/div[5]/div[4]/div/button"

    click_expand_buttons_until_gone(driver, expand_button_xpath)
    # XPath untuk elemen-elemen yang relevan
    common_xpath = "/html/body/div[1]/div[2]/main/div[1]/div[2]/div[5]/div[2]//article/div/div/section[2]/span[1]/div/a"
    rating_path = "/html/body/div[1]/div[2]/main/div[1]/div[2]/div[5]/div[2]//article/section/section[1]/div/span"
    content_path = "/html/body/div[1]/div[2]/main/div[1]/div[2]/div[5]/div[2]//article/section/section[2]/section/div/div[1]/span"
    date_path = "/html/body/div[1]/div[2]/main/div[1]/div[2]/div[5]/div[2]//article/section/section[1]/span/a"

    # Ambil elemen-elemen yang sesuai
    review = driver.find_elements(By.XPATH, common_xpath)
    ratings = driver.find_elements(By.XPATH, rating_path)
    contents = driver.find_elements(By.XPATH, content_path)
    date = driver.find_elements(By.XPATH, date_path)

      # Iterasi untuk setiap elemen ulasan
    for index, rating_element in enumerate(ratings):
        try:
            # Ambil atribut "aria-label" dari elemen rating
            aria_label = rating_element.get_attribute("aria-label")

            # Ekstrak nilai rating menggunakan regex
            match = re.search(r"Rating (\d) out of 5", aria_label)
            rating_number = int(match.group(1)) if match else "Rating tidak valid"
        except Exception as e:
            rating_number = "Rating tidak ditemukan"
            print(f"Kesalahan saat membaca rating untuk elemen {index}: {e}")

    # Pastikan panjang elemen konsisten
    review_count = min(len(review), len(ratings), len(contents), len(date))

    # Iterasi untuk mencetak setiap ulasan
    for index in range(review_count):
        profil = review[index].text if index < len(review) else "N/A"
        dates = date[index].text if index < len(date) else "N/A"
        # rating = rating_number[index].text if index < len(ratings) else "N/A"
        content = contents[index].text if index < len(contents) else "N/A"

        print(f"Review {index + 1}: {profil}")
        print(f"{dates}")
        print(f"  Rating : {rating_number}")
        print(f"  Content: {content}")



    # Tutup browser
    driver.quit()

# Jalankan program
if __name__ == "__main__":
    main()